In [ ]:
############# Flask Modules Setup ##############

from flask import Flask, render_template, url_for, redirect, request, make_response, Response, jsonify
from flask_login import LoginManager, login_user, logout_user, login_required, UserMixin
import MySQLdb

######### Initialize Flask App ##########

app = Flask(__name__)

######### MySQL Database Setup ##########

connection = MySQLdb.connect(host="localhost",
                     user="root",   
                     passwd="1155136999", 
                     db="southwind",   
                     charset='utf8mb4')     

######### Login Manager Setup ##########

login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = 'login'
app.config['SECRET_KEY'] = "lkkajdghdadkglajkgah"


@login_manager.user_loader
def load_user(user_id):
    return User(user_id)

class User(UserMixin):
  def __init__(self,id):
    self.id = id

############ Web Page Routes Setup ###############
    
@app.route("/")
@login_required
def home():
    return render_template('home.html')

@app.route("/login")
def login():
    message = 'Please login in first.'
    return render_template('login.html', message=message)

@app.route('/query_auth')
def query_auth():
    username = request.args.get("username")
    password = request.args.get("password")
    if username == "1155136999@link.cuhk.edu.hk" and password == "1234":
        login_user(User(1))
        message = "Dear " + username + ", welcome to assignment#4. Your login has been granted."
        return make_response(message, 200, )
    else:
        message = 'Attention! Wrong password!'
        return make_response( messagte,401,{'WWW-Authenticate':'Basic realm="Login Required"'})

@app.route("/form_auth",methods=['POST'])
def form_auth():
    username = request.form['email']
    password = request.form['pwd']
    if username == "1155136999@link.cuhk.edu.hk" and password == "1234":
        login_user(User(1))
        message = "Dear " + username + ", welcome to assignment#4. Your login has been granted."
        return render_template('home.html', message=message)
    else:
        message = 'Attention! Wrong password!'
        return render_template('login.html',message=message)

@app.route('/album')
@login_required
def album():
    user = {"name":"Anna"}
    with connection.cursor() as cursor:
        sql = "SELECT * FROM store"
        cursor.execute(sql)
        result = cursor.fetchall()
    final_result = [list(i) for i in result]
    dataset=[]
    dict={}
    for i in final_result:
        dict['Name'] = i[0]
        dict['Place'] = i[1]
        dict['Tel'] = i[2]
        dict['Cheapness'] = i[3]
        dict['Richness'] = i[4]
        dict['Review'] = i[5]
        dict['url'] = i[6]
        dataset.append(dict.copy()) #markers.append(fld.copy())
    return render_template('album.html',album_user=user, dataset=dataset)

@app.route('/logout/')
@login_required
def logout():
    logout_user()
    message = 'Thanks for logging out.'
    return render_template('login.html',message=message)

@app.errorhandler(500)
def internal_error(error):
    message = 'Attention! Wrong password!'
    return render_template('login.html',message=message),500

######### API Endpoints ##########    

@app.route('/api_auth', methods=['POST'])
def api_auth():
    username = request.json['username']
    password = request.json['password']
    if username == "1155136999@link.cuhk.edu.hk" and password == "1234":
        login_user(User(1))
        message = "Dear " + username + ", welcome to assignment#4. Your login has been granted."
        return jsonify({'response':'ok!','message':message})
    else:
        message = 'Attention! Wrong password!'
        return jsonify({'response':'Invalid!','message':'Cannot authenticate.'})
    
@app.route('/api_album', methods=['POST'])
@login_required
def api_album():
    user = {"name":"Anna"}
    with connection.cursor() as cursor:
        sql = "SELECT * FROM store"
        cursor.execute(sql)
        result = cursor.fetchall()
    final_result = [list(i) for i in result]
    dataset=[]
    dict={}
    for i in final_result:
        dict['Name'] = i[0]
        dict['Place'] = i[1]
        dict['Tel'] = i[2]
        dict['Cheapness'] = i[3]
        dict['Richness'] = i[4]
        dict['Review'] = i[5]
        dict['url'] = i[6]
        dataset.append(dict.copy()) #markers.append(fld.copy())
    return jsonify({'Album': dataset})

######### Run Flask Web App at Port 9030 ##########    

if __name__ == '__main__':
    from werkzeug.serving import run_simple
    run_simple('localhost', 9030, app)

 * Running on http://localhost:9030/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Mar/2020 12:11:36] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [06/Mar/2020 12:11:37] "GET /album HTTP/1.1" 200 -
127.0.0.1 - - [06/Mar/2020 12:11:39] "GET /logout/ HTTP/1.1" 200 -
127.0.0.1 - - [06/Mar/2020 12:11:41] "GET /album HTTP/1.1" 302 -
127.0.0.1 - - [06/Mar/2020 12:11:41] "GET /login?next=%2Falbum HTTP/1.1" 200 -
127.0.0.1 - - [06/Mar/2020 12:11:45] "GET /album HTTP/1.1" 302 -
127.0.0.1 - - [06/Mar/2020 12:11:45] "GET /login?next=%2Falbum HTTP/1.1" 200 -
127.0.0.1 - - [06/Mar/2020 12:11:51] "POST /form_auth HTTP/1.1" 200 -
127.0.0.1 - - [06/Mar/2020 12:11:53] "GET /album HTTP/1.1" 200 -
127.0.0.1 - - [06/Mar/2020 12:12:00] "GET /logout/ HTTP/1.1" 200 -
